In [1]:
%%sh
# MC delete this cell at the end, as it is just for interfacing with the underlying linux environment.
ls report/

Montagud2022_Prostate_Cancer.bnd
Montagud2022_Prostate_Cancer.cfg
Montagud2022_Prostate_Cancer.zginml
Montagud2022_interactions_sources.xlsx
Montagud2022_nodes in pathways.xlsx
elife-72626-v2.pdf
supplementary_files


# Introduction to data analysis for natural and social sciences
This notebook contitutes the first part of the exam.

Here the steps of article "Patient-specific Boolean models of signalling networks guide personalised treatments" are retraced and the results reproduced.

## Imports and global settings

In [5]:
import pandas as pd

In [45]:
PATH_REPORT = "report"

EXT_EXCEL = "xlsx"
EXT_TAB = "tsv"

# Prostate Boolean model construction
The Boolean model is constructed starting from information available in literature. Then further pathways are identified by the use of software ROMA and pypath and they are added to the existing network.

The authors collected all the data regarding the network such as nodes, their role, logical rules etc. in the two following Excel files:

In [46]:
f_nodes_pathways = "Montagud2022_nodes in pathways.xlsx"
f_nodes_network = "Montagud2022_interactions_sources.xlsx"

Data are loaded in Pandas dataframe to ease their manipulation.

In [49]:
df_nodes_pathways = pd.read_excel(
    io=f"{PATH_REPORT}/{f_nodes_pathways}",
    header=None,
    names=["node", "pathway"]
)
df_nodes_interactions = pd.read_excel(
    io=f"{PATH_REPORT}/{f_nodes_network}",
    sheet_name="Nodes",
    header=1
)
df_nodes_unique = pd.read_excel(
    io=f"{PATH_REPORT}/{f_nodes_network}",
    sheet_name="Nodes_unique"
)

In [53]:
# MC info.
print(df_nodes_pathways.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   node     133 non-null    object
 1   pathway  133 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB
None


In [51]:
# MC info.
print(df_nodes_interactions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Target node       462 non-null    object
 1   HUGO names        454 non-null    object
 2   Interaction type  462 non-null    object
 3   Source            462 non-null    object
 4   Description       462 non-null    object
 5   Reference: PMID   453 non-null    object
 6   Logical rule      462 non-null    object
dtypes: object(7)
memory usage: 25.4+ KB
None


In [59]:
# One single logical rule is associated to each node, in fact the result of grouping by node and rule is a list of exactly 133 rows.
df_nodes_interactions.groupby(["Target node", "Logical rule"]).count()

,,HUGO names,Interaction type,Source,Description,Reference: PMID
Target node,Logical rule,,,,,
AKT,((HSPs | (PDK1 & PIP3) | PIP3 | (SHH & PIP3)) & !PTCH1),5,5,5,5,5
AMPK,(ATR | HIF1 | AMP_ATP | ATM) & !FGFR3,6,6,6,6,6
AMP_ATP,(!Nutrients),1,1,1,1,1
APAF1,((Caspase8 | BAX | p53 | Bak | HSPs) & !Bcl_XL & !BCL2 & !AKT),8,8,8,8,8
AR,((GLI | EP300 | HSPs | NKX3_1 | EZH2 | NCOA3 | PKC | SMAD | Androgen) & !PTEN & !NCOR1 & !NCOR2 & !MDM2),13,13,13,13,13
...,...,...,...,...,...,...
p21,((p53 | SMAD | HIF1 | ZBTB17) & !TERT & !MYC_MAX & !MDM2 & !AKT & !ERK),9,9,9,9,9
p38,(MAP3K1_3 & !ERK & !GADD45),3,3,3,3,3
p53,((Acidosis | CHK1_2 | p38 | HIF1) & !BCL2 & !MDM2 & !HSPs & !Snail),9,9,9,9,9


In [61]:
# MC info.
print(df_nodes_unique.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Node          121 non-null    object
 1   HGNC symbols  121 non-null    object
 2   unique        121 non-null    object
 3   comments      39 non-null     object
dtypes: object(4)
memory usage: 3.9+ KB
None


In [ ]:
# MC why df_nodes_unique contains only 121 nodes and not 133?
# MC removing input (11 nodes) and output (6 nodes) it should be 133 - 11 - 6 = 116.
# MC probably only inputs are removed, but it is not good anyway because 133 - 11 = 122, where is the one node I am not removing?

Data about nodes are then exported in files with tab-separated values (TSV) format, to import them in Cytoscape.

In [ ]:
# MC useful for export.
f_nodes_pathways.removesuffix(f".{EXT_EXCEL}")
# MC see also https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html